# Load in the confusion matrix

In [1]:
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import gridspec
import numpy as np

In [2]:
region = 'uk'

df = pd.read_csv(f"../outputs/turing-{region}_v02_confusion-data.csv")
labels = pd.read_json(f'/bask/homes/f/fspo1218/amber/data/gbif_{region}/01_{region}_data_numeric_labels.json', typ='series')

In [3]:
df.head()

F_Truth  F_Prediction  G_Truth  G_Prediction  S_Truth  S_Prediction
0        8             8      969           969     2116          2116
1        6             6      972           972      917           917
2       28            28      407           407      160           160
3        1             1       18            18      811           811
4        8             8      853           853     1360          1360

**Key 🗝️**

F= family; 
G= genus; 
S=species

In [4]:
# Calculate accuracy
accuracy = accuracy_score(df['F_Truth'], df['F_Prediction'])
print(f'Family Accuracy: {accuracy:.4f}')

accuracy = accuracy_score(df['G_Truth'], df['G_Prediction'])
print(f'Genus Accuracy: {accuracy:.4f}')

accuracy = accuracy_score(df['S_Truth'], df['S_Prediction'])
print(f'Species Accuracy: {accuracy:.4f}')

Family Accuracy: 0.9523
Genus Accuracy: 0.8995
Species Accuracy: 0.8586


In [5]:
s_lab = labels['species_list']
g_lab = labels['genus_list']
f_lab = labels['family_list']

s_lab = {index: label for index, label in enumerate(s_lab)}
g_lab = {index: label for index, label in enumerate(g_lab)}
f_lab = {index: label for index, label in enumerate(f_lab)}

In [6]:
df['species_truth'] = df['S_Truth'].map(s_lab)
df['species_prediction'] = df['S_Prediction'].map(s_lab)

df['genus_truth'] = df['G_Truth'].map(g_lab)
df['genus_prediction'] = df['G_Prediction'].map(g_lab)

df['family_truth'] = df['F_Truth'].map(f_lab)
df['family_prediction'] = df['F_Prediction'].map(f_lab)

In [7]:
from matplotlib.colors import LogNorm, Normalize
from matplotlib.ticker import MaxNLocator


def confusion_matrix_kg(tax_type="species", dark_mode=False):
    cm = confusion_matrix(df[(tax_type[0]).upper() + '_Truth'], df[(tax_type[0]).upper() + '_Prediction'])
    plt.figure(figsize=(12, 9))
    sns.heatmap(cm, annot=False, fmt='d', cmap='Blues', cbar=True, norm=LogNorm(),
                xticklabels=df[tax_type + '_truth'].unique(),
                yticklabels=df[tax_type + '_truth'].unique())
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title((tax_type).capitalize() + ' Confusion Matrix')
    plt.savefig('./plots/'+tax_type+'_confusion_matrix.png', transparent=True)

In [25]:
def confusion_matrix_kg(tax_type="species", dark_mode=False):
    cm = confusion_matrix(df[(tax_type[0]).upper() + '_Truth'], df[(tax_type[0]).upper() + '_Prediction'])

    plt.figure(figsize=(12, 10))
    if dark_mode:

        sns.heatmap(cm, annot=False, fmt='d', cmap='Blues', cbar=True, norm=LogNorm(),
                    xticklabels=df[tax_type + '_truth'].unique(),
                    yticklabels=df[tax_type + '_truth'].unique(), linewidths=0.5,
                    linecolor='black')
        plt.xlabel('Predicted', color='white')
        plt.ylabel('Actual', color='white')
        plt.title((tax_type).capitalize() + ' Confusion Matrix', color='white')
        plt.xticks(color='white')
        plt.yticks(color='white')
        cbar = plt.gcf().axes[-1]
        cbar.tick_params(labelcolor='white', color='white')
    else:
        sns.heatmap(cm, annot=False, fmt='d', cmap='Blues', cbar=True, norm=LogNorm(),
                    xticklabels=df[tax_type + '_truth'].unique(),
                    yticklabels=df[tax_type + '_truth'].unique(), linewidths=0.5,
                    linecolor='black')
        plt.xlabel('Predicted')
        plt.ylabel('Actual')
        plt.title((tax_type).capitalize() + ' Confusion Matrix')

    plt.tight_layout()
    plt.savefig('./plots/'+tax_type+'_confusion_matrix.png', transparent=True)


In [26]:
confusion_matrix_kg("family", dark_mode=True)

In [10]:
confusion_matrix_kg("genus")

In [11]:
confusion_matrix_kg("species")

In [ ]:
def labelled_heatmap(tax_type):

    conf_matrix = pd.crosstab(df[tax_type + '_truth'], df[tax_type + '_prediction'],
                              rownames=['Actual'], colnames=['Prediction'], dropna=False)

    mapping_dict = df.set_index(tax_type + '_truth')['family_truth'].to_dict()
    labels = conf_matrix.index.map(mapping_dict) #np.random.random_integers(0,5, size=matrix.shape[0])
    lut = dict(zip(set(labels), sns.hls_palette(len(set(labels)), l=0.5, s=0.8)))
    row_colors = (labels).map(lut)

    g=sns.clustermap(conf_matrix, col_cluster=True, row_cluster=True, linewidths=0.1,
                     norm=LogNorm(), dendrogram_ratio=0.1,
                     cmap='Blues', row_colors=[row_colors])
    plt.show()

In [ ]:
labelled_heatmap('genus')

In [ ]:
labelled_heatmap('species')